# Clinical Variant Explorer

This notebook features a custom visualization to explore 

- Author declarative genomics visualizations which adhere to the [Gosling](http://gosling-lang.org/) JSON Specification 
- Render visualizations directly in notebook
- Combine local, remote, and in-memory genomics data sources in visualizations
- Control a Gosling visualization from Python

Start by importing `gosling`. It is a convention to import as `gos` and then access the API through this namespace. 

In [2]:
#! pip install gosling==0.0.8
import gosling as gos

In [3]:
import gosling as gos
import tracks

lollipop_height = 200

clin_var_beddb = gos.beddb(
    "../data/agg/clinvar.bed.beddb",
    genomicFields=[{"index": 1, "name": "start"}, {"index": 2, "name": "end"}],
    valueFields=[{"index": 7, "name": "significance", "type": "nominal"}],
)

categories = [
    "Benign", "Benign/Likely_benign", "Likely_benign", 
    "Uncertain_significance", "Likely_pathogenic", 
    "Pathogenic/Likely_pathogenic", "Pathogenic",
]

colors = ["#5A9F8C", "#5A9F8C", "#029F73", "gray", "#CB96B3", "#CB71A3", "#CB3B8C"]

clin_var_multivec = gos.multivec(
    "../data/agg/density.multires.mv5",
    row="significance",
    column="position",
    value="count",
    categories=categories,
    binSize=4,
)

dy = lollipop_height / len(categories) / 2 # workaround to offset

strips = gos.Track(clin_var_beddb).mark_bar().encode(
    x="start:G",
    y=gos.Y("significance:N", domain=categories, range=[lollipop_height + dy, dy], baseline="Uncertain_significance", legend=True),
    ye=gos.value(lollipop_height/2),
    size=gos.value(1),
    color=gos.value("lightgray"),
    stroke=gos.value("lightgray"),
    strokeWidth=gos.value(1),
    opacity=gos.value(0.3),
).visibility_lt(
    measure="zoomLevel",
    target="mark",
    threshold=100000,
    transitionPadding=100000,
)

points = gos.Track(clin_var_beddb).mark_point().encode(
    x="start:G",
    color=gos.Color("significance:N", domain=categories, range=colors),
    row=gos.Row("significance:N", domain=categories),
    size=gos.value(7),
    opacity=gos.value(0.8),
).visibility_lt(
    measure="zoomLevel",
    target="mark",
    threshold=1000000,
    transitionPadding=1000000,
)

bars = gos.Track(clin_var_multivec).mark_bar().encode(
    x="start:G",
    xe="end:G",
    y=gos.Y("count:Q", axis="none"),
    color=gos.Color("significance:N", domain=categories, range=colors, legend=True)
).visibility_gt(
    measure="zoomLevel",
    target="mark",
    threshold=500000,
    transitionPadding=500000,
)

lolipop = gos.overlay(strips, points, bars).properties(
    width=725,
    height=lollipop_height,
    xDomain=gos.GenomicDomain(chromosome="13", interval=[31500000, 33150000]),
)

# TODO: Need to specify top-level 
view = gos.stack(
    tracks.gene_annotation.properties(id="view1", height=95, width=725),
    lolipop.properties(height=lollipop_height, width=150),
).properties(
    id="view1",
    xDomain=gos.GenomicDomain(chromosome="13", interval=[31500000, 33150000]),
)

view

View({
  alignment: 'stack',
  id: 'view1',
  tracks: (View({
    alignment: 'overlay',
    color: Color({
      domain: ['+', '-'],
      field: 'strand',
      range: ['#7585FF', '#FF8A85'],
      type: 'nominal'
    }),
    height: 95,
    id: 'view1',
    opacity: {'value': 0.8},
    row: Row({
      domain: ['+', '-'],
      field: 'strand',
      type: 'nominal'
    }),
    tracks: (Track({
      data: {'type': 'beddb', 'url': 'https://server.gosling-lang.org/api/v1/tileset_info/?d=gene-annotation', 'genomicFields': [{'index': 1, 'name': 'start'}, {'index': 2, 'name': 'end'}], 'valueFields': [{'index': 5, 'name': 'strand', 'type': 'nominal'}, {'index': 3, 'name': 'name', 'type': 'nominal'}], 'exonIntervalFields': [{'index': 12, 'name': 'start'}, {'index': 13, 'name': 'end'}]},
      dataTransform: [FilterTransform({
        field: 'type',
        oneOf: ['gene'],
        type: 'filter'
      })],
      mark: 'text',
      style: Style({
        dy: -15
      }),
      text: Text({
        field: 'name',
        type: 'nominal'
      }),
      x: X({
        field: 'start',
        type: 'genomic'
      }),
      xe: Xe({
        field: 'end',
        type: 'genomic'
      })
    }), Track({
      data: {'type': 'beddb', 'url': 'https://server.gosling-lang.org/api/v1/tileset_info/?d=gene-annotation', 'genomicFields': [{'index': 1, 'name': 'start'}, {'index': 2, 'name': 'end'}], 'valueFields': [{'index': 5, 'name': 'strand', 'type': 'nominal'}, {'index': 3, 'name': 'name', 'type': 'nominal'}], 'exonIntervalFields': [{'index': 12, 'name': 'start'}, {'index': 13, 'name': 'end'}]},
      dataTransform: [FilterTransform({
        field: 'type',
        oneOf: ['gene'],
        type: 'filter'
      }), FilterTransform({
        field: 'strand',
        oneOf: ['+'],
        type: 'filter'
      })],
      mark: 'triangleRight',
      size: SizeValue({
        value: 15
      }),
      x: X({
        axis: 'top',
        field: 'end',
        type: 'genomic'
      })
    }), Track({
      data: {'type': 'beddb', 'url': 'https://server.gosling-lang.org/api/v1/tileset_info/?d=gene-annotation', 'genomicFields': [{'index': 1, 'name': 'start'}, {'index': 2, 'name': 'end'}], 'valueFields': [{'index': 5, 'name': 'strand', 'type': 'nominal'}, {'index': 3, 'name': 'name', 'type': 'nominal'}], 'exonIntervalFields': [{'index': 12, 'name': 'start'}, {'index': 13, 'name': 'end'}]},
      dataTransform: [FilterTransform({
        field: 'type',
        oneOf: ['gene'],
        type: 'filter'
      }), FilterTransform({
        field: 'strand',
        oneOf: ['-'],
        type: 'filter'
      })],
      mark: 'triangleLeft',
      size: SizeValue({
        value: 15
      }),
      style: Style({
        align: 'right'
      }),
      x: X({
        field: 'start',
        type: 'genomic'
      })
    }), Track({
      data: {'type': 'beddb', 'url': 'https://server.gosling-lang.org/api/v1/tileset_info/?d=gene-annotation', 'genomicFields': [{'index': 1, 'name': 'start'}, {'index': 2, 'name': 'end'}], 'valueFields': [{'index': 5, 'name': 'strand', 'type': 'nominal'}, {'index': 3, 'name': 'name', 'type': 'nominal'}], 'exonIntervalFields': [{'index': 12, 'name': 'start'}, {'index': 13, 'name': 'end'}]},
      dataTransform: [FilterTransform({
        field: 'type',
        oneOf: ['exon'],
        type: 'filter'
      })],
      mark: 'rect',
      size: SizeValue({
        value: 15
      }),
      x: X({
        field: 'start',
        type: 'genomic'
      }),
      xe: Xe({
        field: 'end',
        type: 'genomic'
      })
    }), Track({
      data: {'type': 'beddb', 'url': 'https://server.gosling-lang.org/api/v1/tileset_info/?d=gene-annotation', 'genomicFields': [{'index': 1, 'name': 'start'}, {'index': 2, 'name': 'end'}], 'valueFields': [{'index': 5, 'name': 'strand', 'type': 'nominal'}, {'index': 3, 'name': 'name', 'type': 'nominal'}], 'exonIntervalFields': [{'index': 12, 'name': 'start'}, {'index': 13, 'name': 'end'}]},
      dataT

In [2]:
w = view.widget()
w

GoslingWidget()